In [1]:
import os
import pandas as pd
import polars as pl

In [2]:
def find_path(filename):
    path = "./"

    filepath = ""
    for root,d_names,f_names in os.walk(path):
        if "users.csv" in f_names: break

    filepath = "".join([root, "/", filename])
    return filepath


In [11]:
t

,transaction_id,date,transaction_amount,transaction_category_id,is_blocked,user_id
0,135d739a-71d0-440c-8497-e3e558e39d3d,2022-01-05,29.81,2,True,e671de68-5273-426d-9026-9dbc224d7619
1,8518b69e-54e5-48cf-836d-14524cdc3081,2021-07-11,-240.11,4,True,cc80804a-3ee9-4101-be74-94b412ab18a0
2,9f5d810c-b4d5-4ba7-96d6-cca44aaeb14e,2022-03-01,376.03,3,True,6fb14ef9-9a95-407f-a383-07c753206fcd
3,89decfb4-198c-4d78-8e19-650dbc216b36,2021-07-15,169.01,5,True,71efc6c5-3397-4cae-9057-95047a59d1ff
4,55650689-8a75-4b6c-b76e-b03a74e0746c,2021-08-19,-206.71,2,True,b68247d9-b167-4142-a33e-60d5db1331a1
...,...,...,...,...,...,...
999995,eae6c97e-5ac3-43b5-829b-e7c7e2f390de,2021-11-24,109.71,0,True,ecfb3bd3-9205-4af3-b708-3174a51859df
999996,ebb28546-a097-474b-8b9d-7d3553b1de0a,2022-01-13,135.40,3,True,79c17f6d-3a0d-4557-9ca9-b1979904e3b3
999997,18440cce-9df2-4418-a7bf-ec99f8613aac,2021-08-14,2.68,2,True,39a6a82e-b0ad-43cd-936d-0100213d4df3
999998,6ba7846a-9048-4ff0-bc21-891ca9f9c2fd,2021-07-11,-262.80,3,True,d6dca04b-636e-4aa4-a314-561d636712f0


In [12]:
f = pd.merge(t, u, on="user_id", suffixes=('_trxn', '_user'))
f = f.loc[("is_blocked_user" == False) & ("is_blocked_trxn" == False)]


KeyError: 'False: boolean label can not be used without a boolean index'

## Pandas

In [ ]:
# read data
t = pd.read_csv(find_path("transactions.csv"))
u = pd.read_csv(find_path("users.csv"))

# join and filter
f = pd.merge(t, u, on="user_id", suffixes=('_trxn', '_user'))
f = f.loc[(f.is_blocked_user == False) & (f.is_blocked_trxn == False)]


# group and agregate
res = (
    f
    .groupby("transaction_category_id", as_index=False)
    .agg(
        {
            'transaction_amount': "sum",
            'user_id':            "nunique"
        }
    )
    .rename(columns=
        {
            'transaction_amount': 'sum_amount',
            'user_id':            'num_users'
        }
    )
    .sort_values(by=['sum_amount'], ascending=False)
)

In [8]:
t = pd.read_csv(find_path("transactions.csv"))
u = pd.read_csv(find_path("users.csv"))

# join and filter
f = pd.merge(t, u, on="user_id", suffixes=('_trxn', '_user'))
f = f.loc[(f.is_blocked_user == False) & (f.is_blocked_trxn == False)]


# group and agregate
res = (
    f
    .groupby("transaction_category_id", as_index=False)
    .agg(
        {
            'transaction_amount': "sum",
            'user_id':            "nunique"
        }
    )
    .rename(columns=
        {
            'transaction_amount': 'sum_amount',
            'user_id':            'num_users'
        }
    )
    .sort_values(by=['sum_amount'], ascending=False)
)

In [4]:
res

,transaction_category_id,sum_amount,num_users
2,2,5798.38,752
7,7,1305.54,63
8,8,-44.61,22
1,1,-732.80,734
6,6,-1975.81,200
5,5,-5241.47,408
0,0,-12952.93,538
3,3,-14186.12,693
4,4,-17524.64,596


## Polars

In [5]:
# read data
t = pl.read_csv(find_path("transactions.csv"))
u = pl.read_csv(find_path("users.csv"))


# join, filter, group and agregate
res = (
    t
    .join(u, on="user_id", suffix="_user")
    .filter((pl.col("is_blocked") == False) & (pl.col("is_blocked_user") == False))

    .groupby("transaction_category_id")
    .agg(
        [
            pl.sum     ("transaction_amount").alias("sum_amount"),
            pl.n_unique("user_id")           .alias("num_users"),
        ]
    )
    .sort("sum_amount", reverse=True)
)

In [6]:
res

transaction_category_id,sum_amount,num_users
i64,f64,u32
2,5798.38,752
7,1305.54,63
8,-44.61,22
1,-732.8,734
6,-1975.81,200
5,-5241.47,408
0,-12952.93,538
3,-14186.12,693
4,-17524.64,596
